#For model training with CNN/Vision Transformer

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "L

In [3]:
from datasets import load_dataset
dataset = load_dataset("cj-mills/hagrid-sample-500k-384p", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

hagrid-sample-500k-384p.zip:   0%|          | 0.00/13.4G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset[0]


{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=384x512>,
 'label': 0}

## Label Mapping

In [5]:
label_names = dataset.features['label'].names
print(label_names)

['train_val_call', 'train_val_dislike', 'train_val_fist', 'train_val_four', 'train_val_like', 'train_val_mute', 'train_val_ok', 'train_val_one', 'train_val_palm', 'train_val_peace', 'train_val_peace_inverted', 'train_val_rock', 'train_val_stop', 'train_val_stop_inverted', 'train_val_three', 'train_val_three2', 'train_val_two_up', 'train_val_two_up_inverted']


## Transform the Images

In [10]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

def transform_fn(example):
    imgs = example["image"]

    # Defensive fix: check if list
    if isinstance(imgs, list):
        example["image"] = [transform(img) for img in imgs]
    else:
        example["image"] = transform(imgs)

    return example


## Train-Test Split

In [11]:
dataset.set_transform(transform_fn)

split = dataset.train_test_split(test_size=0.1)
train_dataset = split["train"]
val_dataset = split["test"]

## Wrapping It in a DataLoader

In [12]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


## Viewing the Batch

---



In [13]:
batch = next(iter(train_loader))

print(type(batch["image"]))
print(batch["image"].shape)
print(batch["label"].shape)


<class 'torch.Tensor'>
torch.Size([32, 3, 224, 224])
torch.Size([32])
